In [1]:
import pandas as pd
import torch


In [2]:
df = pd.read_csv('../Datasets/Node2vec/Node2vec_data.csv')
# df_landing = pd.read_csv('../Datasets/Node2vec/landing_page_embeddings.csv')
# df_tags = pd.read_csv('../Datasets/Node2vec/search_tag_embeddings.csv')

In [3]:
print(df.dtypes)

campaign_item_id          int64
advertiser_id             int64
advertiser_name           int64
ext_service_id            int64
creative_id               int64
template_id             float64
channel_id                int64
campaign_budget_usd     float64
media_cost_usd          float64
impressions               int64
clicks                    int64
time                     object
landing_page             object
network_id                int64
keywords                  int64
creative_dimension      float64
channel_Display           int64
channel_Mobile            int64
channel_Search            int64
channel_Social            int64
channel_Video             int64
service_DV360             int64
service_Facebook Ads      int64
service_Google Ads        int64
week_day                  int64
week_end                  int64
dtype: object


## Node Construction

In [4]:
from torch_geometric.data import HeteroData

# List to store graph instances
graphs = []

# Iterate through each record in the dataset
for idx, record in df.iterrows():
    # Create a graph instance for the current record
    graph = HeteroData()

    # === Nodes ===


    # Campaign node
    campaign_feature = torch.tensor([record['campaign_item_id']], dtype=torch.float).unsqueeze(1)
    graph['campaign'].x = campaign_feature


    # Platform node
    # Extract the one-hot encoded platform name columns
    platform = torch.tensor([record['service_DV360'],
                             record['service_Facebook Ads'],
                             record['service_Google Ads']], dtype=torch.float)

    # Combine the one-hot encoded features with any additional features for the platform node
    platform_feature = torch.cat((platform, torch.tensor([record['ext_service_id']], dtype=torch.float))) # Add other platform features

    # Reshape to match dimensionality requirements (N x D, where N=1 and D=number of features)
    graph['platform'].x = platform_feature.view(1,-1)



    # Channel node
    channel = torch.tensor([record['channel_Mobile'],record['channel_Social'],
                            record['channel_Video'],record['channel_Display'],
                            record['channel_Search']], dtype=torch.float)
    channel_feature = torch.cat((channel,torch.tensor([record['channel_id']], dtype=torch.float)))
    channel_feature = channel_feature
    graph['channel'].x = channel_feature.view(1,-1)

    # Network node
    network_feature = torch.tensor([record['network_id']], dtype=torch.float)
    graph['network'].x = network_feature.view(1,-1)

    # Creative node
    creative_feature = torch.tensor([record['creative_id'], record['creative_dimension']], dtype=torch.float)
    graph['creative'].x = creative_feature.view(1,-1)

    # Template node
    template_feature = torch.tensor([record['template_id']], dtype=torch.float)
    graph['template'].x = template_feature.view(1,-1)


    # === Edges ===

    # Define edges specific to this record

    graph['campaign', 'hosted_on', 'platform'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
    graph['campaign', 'uses', 'channel'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
    graph['campaign', 'managed_by', 'network'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
    graph['creative', 'designed_with', 'template'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
    graph['campaign', 'uses', 'creative'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
    graph['platform', 'supports', 'channel'].edge_index  = torch.tensor([[0], [0]], dtype=torch.long)


    # === Target (Clicks) ===
    
    # Assign clicks as the target for the graph
    graph.y = torch.tensor([record['clicks']], dtype=torch.float).view(1,-1)

    # Add the graph instance to the list
    graphs.append(graph)

print(f"Created {len(graphs)} graph instances, one for each dataset record.")


Created 69200 graph instances, one for each dataset record.


#### Notes
.unsqueeze(0) -> Adds a new dimension to tranform from shape [D] to [1,D]

torch.tensor([[0],[0]]) -> first [0] is for source node and next [0] is for target node

torch.tensor(...) -> Converts a Python list or scalar into a pytorch tensor

## Storing graph instances

In [5]:
from torch_geometric.data import InMemoryDataset

class GraphDataset(InMemoryDataset):
    def __init__(self, graphs, transform=None):
        super().__init__(transform=transform)
        self.data, self.slices = self.collate(graphs)

# Create the dataset from the graph instances
dataset = GraphDataset(graphs)
print(f"Dataset contains {len(dataset)} graph instances.")

Dataset contains 69200 graph instances.


#### Notes

This part of the code is used to convert the graph instances to usable dataset.

For fitting the entire dataset, InMemoryDataset is used so that the entire graphs can be fitted.

Efficiently batch graphs during training.

Access individual graphs when needed.






## Spliting the dataset

In [6]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

# Split the dataset
train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_idx, val_idx = train_test_split(train_idx, test_size=0.25, random_state=42)  # 20% test, 20% validation

# Create subsets
train_dataset = dataset[train_idx]
val_dataset = dataset[val_idx]
test_dataset = dataset[test_idx]

print(f"Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")


Train: 41520, Validation: 13840, Test: 13840


## Loading graphs in batches for training and testing

In [7]:
# Define DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"DataLoaders are ready: Train loader size = {len(train_loader)}, Val loader size = {len(val_loader)}, Test loader size = {len(test_loader)}")


DataLoaders are ready: Train loader size = 1298, Val loader size = 433, Test loader size = 433


## GAT Regressor

In [8]:
from torch_geometric.nn import HeteroConv, GATConv, global_mean_pool
import torch.nn.functional as F


class HeteroGATRegressor(torch.nn.Module):
    def __init__(self, input_dims, hidden_dim, output_dim, num_heads):
        super(HeteroGATRegressor, self).__init__()
        # Define GATConv layers for each edge type
        
        # Debugging: Check if input dimensions are correctly initialized
        print("Input Dimensions:", input_dims)

        # Linear layers to align feature dimensions for each node type
        self.feature_transform = torch.nn.ModuleDict({
            key: torch.nn.Linear(input_dim, hidden_dim) for key, input_dim in input_dims.items()
        })

        self.conv1 = HeteroConv({
            ('campaign', 'hosted_on', 'platform'): GATConv(input_dims['campaign'], hidden_dim, heads=num_heads, concat=True, add_self_loops=False),
            ('campaign', 'uses', 'channel'): GATConv(input_dims['campaign'], hidden_dim, heads=num_heads, concat=True, add_self_loops=False),
            ('campaign', 'managed_by', 'network'): GATConv(input_dims['campaign'], hidden_dim, heads=num_heads, concat=True, add_self_loops=False),
            ('creative', 'designed_with', 'template'): GATConv(input_dims['creative'], hidden_dim, heads=num_heads, concat=True, add_self_loops=False),
            ('campaign', 'uses', 'creative'): GATConv(input_dims['campaign'], hidden_dim, heads=num_heads, concat=True, add_self_loops=False),
            ('platform', 'supports', 'channel'): GATConv(input_dims['platform'], hidden_dim, heads=num_heads, concat=True, add_self_loops=False),
        })

        # Debugging: Check the output dimension after concatenation
        print(f"Conv1 Output Dimension: {hidden_dim * num_heads}")

        self.conv2 = HeteroConv({
            ('campaign', 'hosted_on', 'platform'): GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('campaign', 'uses', 'channel'): GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('campaign', 'managed_by', 'network'): GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('creative', 'designed_with', 'template'): GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('campaign', 'uses', 'creative'): GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('platform', 'supports', 'channel'): GATConv(hidden_dim * num_heads, hidden_dim, heads=1, concat=False, add_self_loops=False),
        })

        # Check if the second convolution layer is receiving the correct dimensions
        print(f"Conv2 Output Dimension: {hidden_dim}")

        self.fc1 = torch.nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc2 = torch.nn.Linear(hidden_dim // 2, output_dim)
        

    def forward(self, data):
        
        print("Forward pass")
        print(f"Node feature shapes in data.x_dict:")
        for key, x in data.x_dict.items():
            print(f"{key}: {x.shape}")

        print(f"Edge indices in data.edge_index_dict:")
        for edge_type, edge_index in data.edge_index_dict.items():
            print(f"{edge_type}: {edge_index.shape}")



        # Transform features for each node type to the hidden dimension
        x_dict = {key: F.relu(self.feature_transform[key](x)) for key, x in data.x_dict.items()}
        print("debug1")
        x_dict = self.conv1(data.x_dict, data.edge_index_dict)
        print("debug2")
        x_dict = {key: F.elu(x) for key, x in x_dict.items()}
        print("debug3")

        # Debugging: Check dimensions after first convolution
        for key, x in x_dict.items():
            print(f"After Conv1, {key} node feature shape: {x.shape}")

        # Apply second GAT layer
        x_dict = self.conv2(x_dict, data.edge_index_dict)
        x_dict = {key: F.elu(x) for key, x in x_dict.items()}
        
        # Debugging: Check dimensions after second convolution
        for key, x in x_dict.items():
            print(f"After Conv2, {key} node feature shape: {x.shape}")


        # Pool node features to graph-level features (e.g., using 'campaign' node type)
        x = global_mean_pool(x_dict['campaign'], data['campaign'].batch)
        print(f"Pooled shape: {x.shape}")

        # Apply fully connected layers
        x = F.elu(self.fc1(x))
        print(f"After fc1: {x.shape}")

        x = self.fc2(x)
        print(f"After fc2: {x.shape}")

        return x
    
    

#### Notes

Parameters in __init__
The constructor takes the following parameters:

input_dim: The dimensionality of the input node features.

hidden_dim: The dimensionality of the hidden layers in the GATConv layers.

output_dim: The dimensionality of the output (e.g., 1 for regression).

num_heads: The number of attention heads in the first GATConv layer.

In [9]:
for batch in train_loader:
    print("Node feature shapes:")
    for key, x in batch.x_dict.items():
        print(f"  Node type: {key}, Shape: {x.shape}")
    break


Node feature shapes:
  Node type: campaign, Shape: torch.Size([32, 1])
  Node type: platform, Shape: torch.Size([32, 4])
  Node type: channel, Shape: torch.Size([32, 6])
  Node type: network, Shape: torch.Size([32, 1])
  Node type: creative, Shape: torch.Size([32, 2])
  Node type: template, Shape: torch.Size([32, 1])


In [10]:
# for batch in train_loader:
#     print("Checking consistency between nodes and edges:")
    
#     for edge_type, edge_index in batch.edge_index_dict.items():
#         # Check the number of nodes in the source and target nodes for this edge type
#         src, dst = edge_index
        
#         num_source_nodes = batch[edge_type[0]].x.shape[0]
#         num_target_nodes = batch[edge_type[2]].x.shape[0]
        
#         assert torch.max(src) < num_source_nodes, f"Source node index exceeds batch size for {edge_type}"
#         assert torch.max(dst) < num_target_nodes, f"Target node index exceeds batch size for {edge_type}"
        
#         print(f"Edge type {edge_type} is valid with source node count {num_source_nodes} and target node count {num_target_nodes}.")


In [11]:
input_dims = {
    'campaign': 1,  # campaign_item_id
    'platform': 4,  # service_DV360, service_Facebook Ads, service_Google Ads, ext_service_id
    'channel': 6,   # channel_Mobile, channel_Social, channel_Video, channel_Display, channel_Search, channel_id
    'network': 1,   # network_id
    'creative': 2,  # creative_id, creative_dimension
    'template': 1,  # template_id
}
model = HeteroGATRegressor(input_dims, hidden_dim=32, output_dim=1, num_heads=4)

Input Dimensions: {'campaign': 1, 'platform': 4, 'channel': 6, 'network': 1, 'creative': 2, 'template': 1}
Conv1 Output Dimension: 128
Conv2 Output Dimension: 32


c:\Users\Thinkpad\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch_geometric\nn\conv\hetero_conv.py:76: UserWarning: There exist node types ({'campaign'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.
  warnings.warn(


In [12]:
import torch.optim as optim

# Define loss function and optimizer
criterion = torch.nn.MSELoss()  # For regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [13]:
import torch

def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()

        # Forward pass
        out = model(batch)
        
        # Calculate the loss
        loss = criterion(out, batch.y)
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Accumulate total loss
        total_loss += loss.item()
    
    # Return average loss per batch
    return total_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            # Forward pass
            out = model(batch)
            
            # Calculate the loss
            loss = criterion(out, batch.y)
            total_loss += loss.item()
    
    # Return average loss per batch
    return total_loss / len(loader)

# Training loop
for epoch in range(100):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


Forward pass
Node feature shapes in data.x_dict:
campaign: torch.Size([32, 1])
platform: torch.Size([32, 4])
channel: torch.Size([32, 6])
network: torch.Size([32, 1])
creative: torch.Size([32, 2])
template: torch.Size([32, 1])
Edge indices in data.edge_index_dict:
('campaign', 'hosted_on', 'platform'): torch.Size([2, 32])
('campaign', 'uses', 'channel'): torch.Size([2, 32])
('campaign', 'managed_by', 'network'): torch.Size([2, 32])
('creative', 'designed_with', 'template'): torch.Size([2, 32])
('campaign', 'uses', 'creative'): torch.Size([2, 32])
('platform', 'supports', 'channel'): torch.Size([2, 32])
debug1


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x4 and 1x128)

In [46]:
# Get one batch from the train loader
for batch in train_loader:
    # Get a single graph from the batch
    single_graph = batch

    # Check the node features and edge indices for this graph
    print("Node feature shapes for single graph:")
    for key, x in single_graph.x_dict.items():
        print(f"  Node type: {key}, Shape: {x.shape}")

    print("\nEdge indices for single graph:")
    for edge_type, edge_index in single_graph.edge_index_dict.items():
        print(f"  Edge type: {edge_type}, Edge index shape: {edge_index.shape}")

    # Exit after inspecting the first graph in the batch
    break


Node feature shapes for single graph:
  Node type: campaign, Shape: torch.Size([32, 1])
  Node type: platform, Shape: torch.Size([32, 4])
  Node type: channel, Shape: torch.Size([32, 6])
  Node type: network, Shape: torch.Size([32, 1])
  Node type: creative, Shape: torch.Size([32, 2])
  Node type: template, Shape: torch.Size([32, 1])

Edge indices for single graph:
  Edge type: ('campaign', 'hosted_on', 'platform'), Edge index shape: torch.Size([2, 32])
  Edge type: ('campaign', 'uses', 'channel'), Edge index shape: torch.Size([2, 32])
  Edge type: ('campaign', 'managed_by', 'network'), Edge index shape: torch.Size([2, 32])
  Edge type: ('creative', 'designed_with', 'template'), Edge index shape: torch.Size([2, 32])
  Edge type: ('campaign', 'uses', 'creative'), Edge index shape: torch.Size([2, 32])
  Edge type: ('platform', 'supports', 'channel'), Edge index shape: torch.Size([2, 32])


In [45]:
# Assuming 'model' is your trained GNN model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    # Forward pass for the single graph
    out = model(single_graph)
    print(f"Model output for single graph: {out}")


Forward pass
Node feature shapes in data.x_dict:
campaign: torch.Size([32, 1])
platform: torch.Size([32, 4])
channel: torch.Size([32, 6])
network: torch.Size([32, 1])
creative: torch.Size([32, 2])
template: torch.Size([32, 1])
Edge indices in data.edge_index_dict:
('campaign', 'hosted_on', 'platform'): torch.Size([2, 32])
('campaign', 'uses', 'channel'): torch.Size([2, 32])
('campaign', 'managed_by', 'network'): torch.Size([2, 32])
('creative', 'designed_with', 'template'): torch.Size([2, 32])
('campaign', 'uses', 'creative'): torch.Size([2, 32])
('platform', 'supports', 'channel'): torch.Size([2, 32])
debug1


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x4 and 1x128)

In [47]:
import torch
from torch_geometric.data import HeteroData

# Assume df is your DataFrame containing a single row for a graph
record = df.iloc[0]  # Get the first record for a single graph

# Create a graph instance for the current record
graph = HeteroData()

# === Nodes ===

# Campaign node
campaign_feature = torch.tensor([record['campaign_item_id']], dtype=torch.float).unsqueeze(1)
graph['campaign'].x = campaign_feature

# Platform node
platform = torch.tensor([record['service_DV360'], record['service_Facebook Ads'], record['service_Google Ads']], dtype=torch.float)
platform_feature = torch.cat((platform, torch.tensor([record['ext_service_id']], dtype=torch.float)))
graph['platform'].x = platform_feature.view(-1, 1)

# Channel node
channel = torch.tensor([record['channel_Mobile'], record['channel_Social'], record['channel_Video'], 
                        record['channel_Display'], record['channel_Search']], dtype=torch.float)
channel_feature = torch.cat((channel, torch.tensor([record['channel_id']], dtype=torch.float)))
graph['channel'].x = channel_feature.view(-1, 1)

# Network node
network_feature = torch.tensor([record['network_id']], dtype=torch.float)
graph['network'].x = network_feature.view(-1, 1)

# Creative node
creative_feature = torch.tensor([record['creative_id'], record['creative_dimension']], dtype=torch.float)
graph['creative'].x = creative_feature.view(-1, 1)

# Template node
template_feature = torch.tensor([record['template_id']], dtype=torch.float)
graph['template'].x = template_feature.view(-1, 1)

# === Edges ===
graph['campaign', 'hosted_on', 'platform'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
graph['campaign', 'uses', 'channel'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
graph['campaign', 'managed_by', 'network'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
graph['creative', 'designed_with', 'template'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
graph['campaign', 'uses', 'creative'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)
graph['platform', 'supports', 'channel'].edge_index = torch.tensor([[0], [0]], dtype=torch.long)

# === Target (Clicks) ===
graph.y = torch.tensor([record['clicks']], dtype=torch.float)

# Print the node features and edge indices for the single graph
print("Node feature shapes for single graph:")
for node_type, x in graph.x_dict.items():
    print(f"  Node type: {node_type}, Shape: {x.shape}")

print("\nEdge indices for single graph:")
for edge_type, edge_index in graph.edge_index_dict.items():
    print(f"  Edge type: {edge_type}, Edge index shape: {edge_index.shape}")

# Pass this single graph through your model later (for testing)


Node feature shapes for single graph:
  Node type: campaign, Shape: torch.Size([1, 1])
  Node type: platform, Shape: torch.Size([4, 1])
  Node type: channel, Shape: torch.Size([6, 1])
  Node type: network, Shape: torch.Size([1, 1])
  Node type: creative, Shape: torch.Size([2, 1])
  Node type: template, Shape: torch.Size([1, 1])

Edge indices for single graph:
  Edge type: ('campaign', 'hosted_on', 'platform'), Edge index shape: torch.Size([2, 1])
  Edge type: ('campaign', 'uses', 'channel'), Edge index shape: torch.Size([2, 1])
  Edge type: ('campaign', 'managed_by', 'network'), Edge index shape: torch.Size([2, 1])
  Edge type: ('creative', 'designed_with', 'template'), Edge index shape: torch.Size([2, 1])
  Edge type: ('campaign', 'uses', 'creative'), Edge index shape: torch.Size([2, 1])
  Edge type: ('platform', 'supports', 'channel'), Edge index shape: torch.Size([2, 1])


In [50]:
import torch
import torch_geometric
from torch_geometric.nn import HeteroConv, GCNConv, to_hetero

# Define your GNN model (assuming it's already defined)
class HeteroGNN(torch.nn.Module):
    def __init__(self, input_dims, hidden_dim):
        super(HeteroGNN, self).__init__()
        # Linear layers to transform node features
        self.feature_transform = torch.nn.ModuleDict({
            key: torch.nn.Linear(input_dim, hidden_dim) for key, input_dim in input_dims.items()
        })
        
        # Convolution layers
        self.conv1 = HeteroConv(
            {('campaign', 'hosted_on', 'platform'): GCNConv(hidden_dim, hidden_dim, add_self_loops=False),
             ('campaign', 'uses', 'channel'): GCNConv(hidden_dim, hidden_dim,add_self_loops=False),
             ('campaign', 'managed_by', 'network'): GCNConv(hidden_dim, hidden_dim, add_self_loops=False),
             ('creative', 'designed_with', 'template'): GCNConv(hidden_dim, hidden_dim, add_self_loops=False),
             ('campaign', 'uses', 'creative'): GCNConv(hidden_dim, hidden_dim, add_self_loops=False),
             ('platform', 'supports', 'channel'): GCNConv(hidden_dim, hidden_dim, add_self_loops=False)
            }, aggr='mean'
        )
        self.fc_out = torch.nn.Linear(hidden_dim, 1)  # Output layer to predict clicks

    def forward(self, data):
        # Transform node features
        x_dict = {key: torch.relu(self.feature_transform[key](x)) for key, x in data.x_dict.items()}
        
        # Apply first GNN layer (convolution)
        x_dict = self.conv1(x_dict, data.edge_index_dict)

        # Output layer
        out = self.fc_out(x_dict['campaign'])  # Assuming 'campaign' is the target
        
        return out

# Now, instantiate the model and pass the single graph
input_dims = {
    'campaign': 1, 
    'platform': 4, 
    'channel': 6, 
    'network': 1, 
    'creative': 2, 
    'template': 1
}

hidden_dim = 128  # Define your hidden dimension for GNN
model = HeteroGNN(input_dims, hidden_dim)

# Pass the single graph through the model
graph = graph.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))  # Move to GPU if available

# Forward pass
output = model(graph)

# Print the output for the single graph
print("Model output (predicted clicks):", output)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x1 and 4x128)